In [20]:
from sagemaker import RandomCutForest


In [2]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import json

In [3]:
aws_role = get_execution_role()


sm_role = sagemaker.get_execution_role()

aws_region = boto3.Session().region_name
sm_session = sagemaker.Session()

print(f"Using Role: {sm_role}")
s3_bucket = sm_session.default_bucket()
s3_bucket

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Using Role: arn:aws:iam::287758680514:role/service-role/AmazonSageMaker-ExecutionRole-20231027T004238


'sagemaker-us-east-1-287758680514'

In [4]:
import pandas as pd
url = 'https://raw.githubusercontent.com/numenta/NAB/master/data/realKnownCause/ambient_temperature_system_failure.csv'
df = pd.read_csv(url, sep=',')

In [5]:
df

,timestamp,value
0,2013-07-04 00:00:00,69.880835
1,2013-07-04 01:00:00,71.220227
2,2013-07-04 02:00:00,70.877805
3,2013-07-04 03:00:00,68.959400
4,2013-07-04 04:00:00,69.283551
...,...,...
7262,2014-05-28 11:00:00,72.370206
7263,2014-05-28 12:00:00,72.172956
7264,2014-05-28 13:00:00,72.046565
7265,2014-05-28 14:00:00,71.825226


In [6]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [7]:
df['year'] = df['timestamp'].apply(lambda x: x.year)

In [8]:
df['month'] = df['timestamp'].apply(lambda x: x.month)

In [9]:
df['day'] = df['timestamp'].apply(lambda x: x.day)

In [10]:
df['hour'] = df['timestamp'].apply(lambda x: x.hour)

In [11]:
df['minute'] = df['timestamp'].apply(lambda x: x.minute)

In [12]:
df

,timestamp,value,year,month,day,hour,minute
0,2013-07-04 00:00:00,69.880835,2013,7,4,0,0
1,2013-07-04 01:00:00,71.220227,2013,7,4,1,0
2,2013-07-04 02:00:00,70.877805,2013,7,4,2,0
3,2013-07-04 03:00:00,68.959400,2013,7,4,3,0
4,2013-07-04 04:00:00,69.283551,2013,7,4,4,0
...,...,...,...,...,...,...,...
7262,2014-05-28 11:00:00,72.370206,2014,5,28,11,0
7263,2014-05-28 12:00:00,72.172956,2014,5,28,12,0
7264,2014-05-28 13:00:00,72.046565,2014,5,28,13,0
7265,2014-05-28 14:00:00,71.825226,2014,5,28,14,0


In [13]:
anomaly_points = [
        ["2013-12-10 06:25:00.000000","2013-12-12 05:35:00.000000"],
        ["2013-12-15 17:50:00.000000","2013-12-17 17:00:00.000000"],
        ["2014-01-27 14:20:00.000000","2014-01-29 13:30:00.000000"],
        ["2014-02-07 14:55:00.000000","2014-02-09 14:05:00.000000"]
]

In [14]:
anomaly_points

[['2013-12-10 06:25:00.000000', '2013-12-12 05:35:00.000000'],
 ['2013-12-15 17:50:00.000000', '2013-12-17 17:00:00.000000'],
 ['2014-01-27 14:20:00.000000', '2014-01-29 13:30:00.000000'],
 ['2014-02-07 14:55:00.000000', '2014-02-09 14:05:00.000000']]

In [15]:
df['anomaly'] = 0

In [16]:
for start,end in anomaly_points:
    print (start, end)
    df.loc[  ((df['timestamp'] >= start) & (df['timestamp'] <= end)), ['anomaly']] = 1  

2013-12-10 06:25:00.000000 2013-12-12 05:35:00.000000
2013-12-15 17:50:00.000000 2013-12-17 17:00:00.000000
2014-01-27 14:20:00.000000 2014-01-29 13:30:00.000000
2014-02-07 14:55:00.000000 2014-02-09 14:05:00.000000


In [18]:
df[df['anomaly'] == 1].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 190 entries, 3420 to 4891
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  190 non-null    datetime64[ns]
 1   value      190 non-null    float64       
 2   year       190 non-null    int64         
 3   month      190 non-null    int64         
 4   day        190 non-null    int64         
 5   hour       190 non-null    int64         
 6   minute     190 non-null    int64         
 7   anomaly    190 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(6)
memory usage: 13.4 KB


In [19]:
df.index = df['timestamp']

In [20]:
df.drop(['timestamp'], axis=1, inplace=True)

In [21]:
df

,value,year,month,day,hour,minute,anomaly
timestamp,,,,,,,
2013-07-04 00:00:00,69.880835,2013,7,4,0,0,0
2013-07-04 01:00:00,71.220227,2013,7,4,1,0,0
2013-07-04 02:00:00,70.877805,2013,7,4,2,0,0
2013-07-04 03:00:00,68.959400,2013,7,4,3,0,0
2013-07-04 04:00:00,69.283551,2013,7,4,4,0,0
...,...,...,...,...,...,...,...
2014-05-28 11:00:00,72.370206,2014,5,28,11,0,0
2014-05-28 12:00:00,72.172956,2014,5,28,12,0,0
2014-05-28 13:00:00,72.046565,2014,5,28,13,0,0


In [17]:
# !conda install -y holoviews

In [18]:
import seaborn as sns
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

Matplotlib is building the font cache; this may take a moment.


ModuleNotFoundError: No module named 'holoviews'

In [23]:
from sagemaker import RandomCutForest

In [31]:
s3_bucket, prefix

('sagemaker-us-east-1-287758680514', 'randomCut')

In [32]:
prefix='randomaut'
rcf = RandomCutForest(role=sm_role,
                      train_instance_count=1,
                      train_instance_type='ml.m4.xlarge',
                      data_location=f's3://{s3_bucket}/{prefix}/',
                      output_path=f's3://{s3_bucket}/{prefix}/output')

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [33]:
rcf.fit(rcf.record_set(df['value'].to_numpy().reshape(-1,1)))

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: randomcutforest-2023-12-08-05-33-50-347


2023-12-08 05:33:50 Starting - Starting the training job......
2023-12-08 05:34:26 Starting - Preparing the instances for training......
2023-12-08 05:35:51 Downloading - Downloading input data...
2023-12-08 05:36:21 Training - Downloading the training image..................
2023-12-08 05:39:07 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
/opt/amazon/lib/python3.8/site-packages/mxnet/model.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if num_device is 1 and 'dist' not in kvstore:
/opt/amazon/lib/python3.8/site-packages/scipy/optimize/_shgo.py:495: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if cons['type'] is 'ineq':
/opt/amazon/lib/python3.8/site-packages/scipy/optimize/_shgo.py:743: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if len(self.X_min) is not 0:
[12/08/2023 05:39:31 INFO 139853672802112] Reading default conf

In [34]:
rcf_interference = rcf.deploy(initial_instance_count=1, ins)


SyntaxError: positional argument follows keyword argument (1293824993.py, line 1)

In [36]:
from sagemaker.serverless import ServerlessInferenceConfig

In [37]:
serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=2048,
    max_concurrency=5
)

In [38]:
rcf.deploy(initial_instance_count=1, serverless_inference_config=serverless_config)

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating model with name: randomcutforest-2023-12-08-05-56-39-318
INFO:sagemaker:Creating endpoint-config with name randomcutforest-2023-12-08-05-56-39-318
INFO:sagemaker:Creating endpoint with name randomcutforest-2023-12-08-05-56-39-318


-----!

In [39]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer


In [41]:
rcf_interference.serializer = CSVSerializer()
rcf_interference.deserializer = JSONDeserializer()

NameError: name 'rcf_interference' is not defined

In [42]:
sm_session.

In [43]:
def list_sagemaker_endpoints():
    client = boto3.client('sagemaker')
    response = client.list_endpoints(
        SortBy='Name',
        SortOrder='Ascending',
        # MaxResults and NextToken can be used for pagination
        # MaxResults=20,
        # NextToken='string'
    )
    return response['Endpoints']

In [46]:
a =list_sagemaker_endpoints()

In [47]:
a

[{'EndpointName': 'randomcutforest-2023-12-08-05-56-39-318',
  'EndpointArn': 'arn:aws:sagemaker:us-east-1:287758680514:endpoint/randomcutforest-2023-12-08-05-56-39-318',
  'CreationTime': datetime.datetime(2023, 12, 8, 5, 56, 40, 553000, tzinfo=tzlocal()),
  'LastModifiedTime': datetime.datetime(2023, 12, 8, 5, 59, 20, 714000, tzinfo=tzlocal()),
  'EndpointStatus': 'InService'}]

In [49]:
machine_data_numpy = df.value.to_numpy().reshape(-1, 1)

In [50]:
machine_data_numpy

array([[69.88083514],
       [71.22022706],
       [70.87780496],
       ...,
       [72.04656545],
       [71.82522648],
       [72.58408858]])

In [56]:
# Initialize the SageMaker Runtime client
sagemaker_runtime = boto3.client('sagemaker-runtime')

# Example data to be predicted
data = {"features": [5.1, 3.5, 1.4, 0.2]}
serialized_data = json.dumps(machine_data_numpy.tolist())


In [58]:

# Invoke the endpoint
response = sagemaker_runtime.invoke_endpoint(
    EndpointName='randomcutforest-2023-12-08-05-56-39-318',
    ContentType='application/json',
    Body=serialized_data
)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from model with message "unable to evaluate payload provided". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/randomcutforest-2023-12-08-05-56-39-318 in account 287758680514 for more information.